In [ ]:
#!/usr/bin/env python
from __future__ import print_function

import skimage as skimage
from skimage import transform, color, exposure
from skimage.viewer import ImageViewer
import random
import sys
import os
from random import choice
import numpy as np
from collections import deque
import time
import tensorflow as tf
import json
from tensorflow.keras import models


from tensorflow.keras import layers
from tensorflow.keras import optimizers
from keras import backend as K
import vizdoom as vzd
from vizdoom import DoomGame, ScreenResolution
from vizdoom import *
import itertools as it
from time import sleep
from time import time


In [ ]:
if len(tf.config.experimental.list_physical_devices('GPU')) > 1:
    print("GPU available")
    DEVICE = "/gpu:0"
else:
    print("No GPU available")
    DEVICE = "/cpu:0"

In [ ]:
def preprocessImg(img, size):

    img = np.rollaxis(img, 0, 2) # It becomes (640, 480, 3)
    img = skimage.transform.resize(img, size)
    img = skimage.color.rgb2gray(img) 

    return img

In [ ]:
teststart = time()

In [ ]:
def actor_network(input_shape, action_size, learning_rate):
        """Actor Network for A2C
        """

        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(32, 8, strides=(4,4), input_shape=(input_shape)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation('relu'))
        model.add(tf.keras.layers.Conv2D(64, 4, strides=(2,2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Conv2D(64, 3, 3, padding="same"))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(64))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation('relu'))
        model.add(tf.keras.layers.Dense(32))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Dense(action_size, activation='softmax'))

        adam = tf.keras.optimizers.Adam(lr=learning_rate)
        model.compile(loss='categorical_crossentropy',optimizer=adam)
        print("Actor Model")
        model.summary()
        return model

In [ ]:
def critic_network(input_shape, value_size, learning_rate):
        """Critic Network for A2C
        """

        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(32, 8, strides=(4,4), input_shape=(input_shape)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Conv2D(64, 4, strides=(2,2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Conv2D(64, 3, 3, padding="same"))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(64))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Dense(32))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Dense(value_size, activation='linear'))

        adam = tf.keras.optimizers.Adam(lr=learning_rate)
        model.compile(loss='mse',optimizer=adam)
        print("Critic Model")
        model.summary()
        return model

In [ ]:
   
class A2CAgent:

    def __init__(self, state_size, action_size):
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1
        self.observe = 0
        self.frame_per_action = 4

        # These are hyper parameters for the Policy Gradient
        self.gamma = 0.99
        self.actor_lr = 0.0001
        self.critic_lr = 0.0001

        # Model for policy and critic network
        self.actor = None
        self.critic = None

        # lists for the states, actions and rewards
        self.states, self.actions, self.rewards = [], [], []

        # Performance Statistics
        self.stats_window_size= 50 # window size for computing rolling statistics
        self.mavg_score = [] # Moving Average of Survival Time
        self.var_score = [] # Variance of Survival Time
        self.mavg_ammo_left = [] # Moving Average of Ammo used
        self.mavg_kill_counts = [] # Moving Average of Kill Counts

    # using the output of policy network, pick action stochastically (Stochastic Policy)
    def get_action(self, state):
        policy = self.actor.predict(state).flatten()
        return np.random.choice(self.action_size, 1, p=policy)[0], policy

    # Instead agent uses sample returns for evaluating policy
    # Use TD(1) i.e. Monte Carlo updates 
    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, len(rewards))):
            if rewards[t] != 0:
                running_add = 0
            running_add = running_add * self.gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    # save <s, a ,r> of each step
    def append_sample(self, state, action, reward):
        self.states.append(state)
        self.rewards.append(reward)
        self.actions.append(action)

    # update policy network every episode
    def train_model(self):
        episode_length = len(self.states)

        discounted_rewards = self.discount_rewards(self.rewards)
        # Standardized discounted rewards
        discounted_rewards -= np.mean(discounted_rewards) 
        if np.std(discounted_rewards):
            discounted_rewards /= np.std(discounted_rewards)
        else:
            self.states, self.actions, self.rewards = [], [], []
            print ('std = 0!')
            return 0

        update_inputs = np.zeros(((episode_length,) + self.state_size)) # Episode_lengthx64x64x4

        # Episode length is like the minibatch size in DQN
        for i in range(episode_length):
            update_inputs[i,:,:,:] = self.states[i]

        # Prediction of state values for each state appears in the episode
        values = self.critic.predict(update_inputs)

        # Similar to one-hot target but the "1" is replaced by Advantage Function i.e. discounted_rewards R_t - Value
        advantages = np.zeros((episode_length, self.action_size))

        for i in range(episode_length):
            advantages[i][self.actions[i]] = discounted_rewards[i] - values[i]
        
        actor_loss = self.actor.fit(update_inputs, advantages, epochs=1, verbose=0)
        critic_loss = self.critic.fit(update_inputs, discounted_rewards, epochs=1, verbose=0)

        self.states, self.actions, self.rewards = [], [], []

        return actor_loss.history['loss'], critic_loss.history['loss']


    def shape_reward(self, r_t, misc, prev_misc, t):
        
        # Check any kill count
        if (misc[0] > prev_misc[0]):
            r_t = r_t + 300

        if (misc[1] < prev_misc[1]): # Use ammo
            r_t = r_t - 10

        if (misc[2] < prev_misc[2]): # Loss HEALTH
            r_t = r_t - 0.1

        return r_t

    def save_model(self, name):
        self.actor.save_weights(name + "_actor.h5", overwrite=True)
        self.critic.save_weights(name + "_critic.h5", overwrite=True)

    def load_model(self, name):
        self.actor.load_weights(name + "_actor.h5", overwrite=True)
        self.critic.load_weights(name + "_critic.h5", overwrite=True)

In [ ]:
if __name__ == "__main__":


    # Avoid Tensorflow eats up GPU memory
    #config = tf.ConfigProto()
    #config.gpu_options.allow_growth = True
    #sess = tf.Session(config=config)
    #K.set_session(sess)

    with tf.device(DEVICE):
        game = vzd.DoomGame()
        game.load_config("/home/spillingvoid/Downloads/programs/ViZDoom/scenarios/rocket_basic.cfg")
        game.add_available_game_variable(vzd.GameVariable.KILLCOUNT)
        game.add_available_game_variable(vzd.GameVariable.AMMO5)
        game.add_available_game_variable(vzd.GameVariable.HEALTH)
        game.set_window_visible(True)
        game.set_mode(vzd.Mode.PLAYER)
        game.set_screen_format(vzd.ScreenFormat.GRAY8)
        game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
        game.get_available_buttons_size()
        game.init()

    # Maximum number of episodes
        max_episodes = 128 #1000000

        game.new_episode()
        game_state = game.get_state()
        misc = game_state.game_variables  # [KILLCOUNT, AMMO, HEALTH]
        prev_misc = misc

        action_size = game.get_available_buttons_size()

        img_rows , img_cols = 30, 45
    # Convert image into Black and white
        img_channels = 4 # We stack 4 frames

        state_size = (img_rows, img_cols, img_channels)
        agent = A2CAgent(state_size, action_size)
        agent.actor = actor_network(state_size, action_size, agent.actor_lr)
        agent.critic = critic_network(state_size, agent.value_size, agent.critic_lr)

    # Start training
        GAME = 0
        t = 0
        max_life = 0 # Maximum episode life (Proxy for agent performance)

    # Buffer to compute rolling statistics 
        life_buffer, ammo_buffer, kills_buffer = [], [], [] 

        for i in range(max_episodes):

            game.new_episode()
            game_state = game.get_state()
            misc = game_state.game_variables 
            prev_misc = misc

            x_t = game_state.screen_buffer # 480 x 640
            x_t = preprocessImg(x_t, size=(img_rows, img_cols))
            s_t = np.stack(([x_t]*4), axis=2) # It becomes 64x64x4
            s_t = np.expand_dims(s_t, axis=0) # 1x64x64x4

            life = 0 # Episode life

            while not game.is_episode_finished():

                loss = 0 # Training Loss at each update
                r_t = 0 # Initialize reward at time t
                a_t = np.zeros([action_size]) # Initialize action at time t

                x_t = game_state.screen_buffer
                x_t = preprocessImg(x_t, size=(img_rows, img_cols))
                x_t = np.reshape(x_t, (1, img_rows, img_cols, 1))
                s_t = np.append(x_t, s_t[:, :, :, :3], axis=3)
                
            # Sample action from stochastic softmax policy
                action_idx, policy  = agent.get_action(s_t)
                a_t[action_idx] = 1 

                a_t = a_t.astype(int)
                game.set_action(a_t.tolist())
                skiprate = agent.frame_per_action # Frame Skipping = 4
                game.advance_action(skiprate)

                r_t = game.get_last_reward()  # Each frame we get reward of 0.1, so 4 frames will be 0.4
            # Check if episode is terminated
                is_terminated = game.is_episode_finished()

                if (is_terminated):
                # Save max_life
                    if (life > max_life):
                        max_life = life 
                    life_buffer.append(life)
                    ammo_buffer.append(misc[1])
                    kills_buffer.append(misc[0])
                    print ("Episode Finish ", prev_misc, policy)
                else:
                    life += 1
                    game_state = game.get_state()  # Observe again after we take the action
                    misc = game_state.game_variables

            # Reward Shaping
                r_t = agent.shape_reward(r_t, misc, prev_misc, t)

            # Save trajactory sample <s, a, r> to the memory
                agent.append_sample(s_t, action_idx, r_t)

            # Update the cache
                t += 1
                prev_misc = misc

                if (is_terminated and t > agent.observe):
                # Every episode, agent learns from sample returns
                    loss = agent.train_model()

            # Save model every 10000 iterations
                if t % 32 == 0: #10000 == 0:
                    print("Save model")
                    agent.save_model("/home/spillingvoid/Downloads/programs/Doom/model/a2c")

                state = ""
                if t <= agent.observe:
                    state = "Observe mode"
                else:
                    state = "Train mode"

                if (is_terminated):

                # Print performance statistics at every episode end
                    print("TIME", t, "/ GAME", GAME, "/ STATE", state, "/ ACTION", action_idx, "/ REWARD", r_t, "/ LIFE", max_life, "/ LOSS", loss)

                # Save Agent's Performance Statistics
                    if GAME % agent.stats_window_size == 0 and t > agent.observe: 
                        print("Update Rolling Statistics")
                        agent.mavg_score.append(np.mean(np.array(life_buffer)))
                        agent.var_score.append(np.var(np.array(life_buffer)))
                        agent.mavg_ammo_left.append(np.mean(np.array(ammo_buffer)))
                        agent.mavg_kill_counts.append(np.mean(np.array(kills_buffer)))

                    # Reset rolling stats buffer
                        life_buffer, ammo_buffer, kills_buffer = [], [], [] 

                    # Write Rolling Statistics to file
                        with open("/home/spillingvoid/Downloads/programs/Doom/statistics/a2c_stats.txt", "a+") as stats_file:
                            stats_file.write('Game: ' + str(GAME) + '\n')
                            stats_file.write('Max Score: ' + str(max_life) + '\n')
                            stats_file.write('mavg_score: ' + str(agent.mavg_score) + '\n')
                            stats_file.write('var_score: ' + str(agent.var_score) + '\n')
                            stats_file.write('mavg_ammo_left: ' + str(agent.mavg_ammo_left) + '\n')
                            stats_file.write('mavg_kill_counts: ' + str(agent.mavg_kill_counts) + '\n')
        
        # Episode Finish. Increment game count
            GAME += 1

In [ ]:
print("Training complete")
endtime = time()
print(" Test Time elapsed: %.2f minutes" % ((endtime - teststart) / 60.0))